In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv('Resources/lending_data.csv')

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns='loan_status')

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lrmodel = LogisticRegression(random_state=1)

# Fit the model using training data
lrmodel.fit(X_train,y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = lrmodel.predict(X_test)
results_df = pd.DataFrame({'prediction':predictions,'actual':y_test}).reset_index(drop=True)
results_df.head(20)

,prediction,actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

In [11]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18679,    80],
       [   67,   558]], dtype=int64)

In [12]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** WRITE YOUR ANSWER HERE!

The logistic regression model predicts risks well overall. However, the model is more well equipped to predict healthy loans as there is a higher percentage of false negatives (higher percentage of occurrences where a high risk loan was classified as healthy).

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversampler_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
ro_model = oversampler_model.fit(X_train,y_train)

In [16]:
# Count the distinct values of the resampled labels data
X_train_resampled, y_train_resampled = ro_model.fit_resample(X_train,y_train)
np.bincount(y_train_resampled)

array([56277, 56277], dtype=int64)

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [17]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_reg_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
lr_model = logistic_reg_model.fit(X_train_resampled,y_train_resampled)

# Make a prediction using the testing data
resampled_predictions = lr_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, resampled_predictions)

0.9959744975744975

In [19]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, resampled_predictions)

array([[18668,    91],
       [    2,   623]], dtype=int64)

In [20]:
# Print the classification report for the model
classification_report(y_test, resampled_predictions)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     18759\n           1       0.87      1.00      0.93       625\n\n    accuracy                           1.00     19384\n   macro avg       0.94      1.00      0.96     19384\nweighted avg       1.00      1.00      1.00     19384\n'

### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** YOUR ANSWER HERE!

While the regression model fit with oversampled data was accurate overall, it was much more accurate in predicting healthy loans. The rate of false negatives (predicting a healthy loan when it was actually high risk) was much lower than the rate of false positives. And the rate of false negatives was much lower than in the logistic regression model with the original data.